In [1]:
import pandas as pd

In [25]:
from nltk.corpus import stopwords

In [4]:
df = pd.read_csv('row_cleaned.csv', index_col = 0)

In [5]:
df.head()

,review,sentiment
Id,,
0,1974 teenager martha moxley maggie grace move...,1
1,ok so really like kris kristofferson usual eas...,0
2,spoiler do not read this if you think about wa...,0
3,hi for all people who have seen this wonderful...,1
4,recently bought dvd forgetting just how much ...,0


In [6]:
# This function converts a text to a sequence of words.
def review_wordlist(review, remove_stopwords=False):
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # 3. Converting to lower case and splitting
    words = review_text.lower().split()
    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)

In [7]:
import nltk.data

In [8]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:


# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    # This returns the list of lists
    return sentences



In [56]:
X = df['review'].values
Y = df['sentiment'].values
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 20)

In [61]:
train = pd.DataFrame({'review': X_train, 'sentiment':Y_train})
test = pd.DataFrame({'review': X_test, 'sentiment':Y_test})

In [62]:
train.head()

,review,sentiment
0,im large scarred heterosexual male ex bouncer ...,1
1,watched this movie about six years ago recent...,0
2,obviously it seems many people really enjoyed ...,0
3,whats happening rgv he seems repeat himself ev...,0
4,have seen poor movies time but this really ta...,0


In [63]:
from bs4 import BeautifulSoup 
import re

sentences = []
print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_sentences(review, tokenizer)

Parsing sentences from training set


/home/erick/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/erick/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [64]:
len(sentences)

35000

In [65]:


# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



In [66]:
# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-11-08 14:29:06,993 : INFO : collecting all words and their counts
2018-11-08 14:29:06,993 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model....


2018-11-08 14:29:07,297 : INFO : PROGRESS: at sentence #10000, processed 1740785 words, keeping 59796 word types
2018-11-08 14:29:07,585 : INFO : PROGRESS: at sentence #20000, processed 3455216 words, keeping 83128 word types
2018-11-08 14:29:07,882 : INFO : PROGRESS: at sentence #30000, processed 5202230 words, keeping 101327 word types
2018-11-08 14:29:08,031 : INFO : collected 109191 word types from a corpus of 6054816 raw words and 35000 sentences
2018-11-08 14:29:08,032 : INFO : Loading a fresh vocabulary
2018-11-08 14:29:08,086 : INFO : effective_min_count=40 retains 10220 unique words (9% of original 109191, drops 98971)
2018-11-08 14:29:08,087 : INFO : effective_min_count=40 leaves 5600211 word corpus (92% of original 6054816, drops 454605)
2018-11-08 14:29:08,113 : INFO : deleting the raw counts dictionary of 109191 items
2018-11-08 14:29:08,117 : INFO : sample=0.001 downsamples 52 most-common words
2018-11-08 14:29:08,117 : INFO : downsampling leaves estimated 4847355 word co

In [18]:
model.wv.most_similar('man')

/home/erick/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.621584415435791),
 ('mans', 0.6111232042312622),
 ('lady', 0.514820396900177),
 ('boy', 0.4838256239891052),
 ('guy', 0.4823318421840668),
 ('men', 0.4685211181640625),
 ('soldier', 0.4443958103656769),
 ('lad', 0.43793267011642456),
 ('priest', 0.43287044763565063),
 ('lawyer', 0.4299907684326172)]

In [19]:
model.wv.most_similar('awful')

/home/erick/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.8090012669563293),
 ('dreadful', 0.7749406099319458),
 ('horrible', 0.774091362953186),
 ('atrocious', 0.7458703517913818),
 ('horrendous', 0.7247278690338135),
 ('abysmal', 0.7203051447868347),
 ('lousy', 0.7183014154434204),
 ('horrid', 0.7174729108810425),
 ('appalling', 0.6920179128646851),
 ('bad', 0.6690243482589722)]

In [20]:
model.wv.syn0.shape

/home/erick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(12826, 300)

In [21]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [22]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [27]:
import numpy as np
# Calculating average feature vector for training set
clean_train_reviews = []
for review in df['review']:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

/home/erick/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/erick/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 50000


/home/erick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 1000 of 50000
Review 2000 of 50000
Review 3000 of 50000
Review 4000 of 50000
Review 5000 of 50000
Review 6000 of 50000
Review 7000 of 50000
Review 8000 of 50000
Review 9000 of 50000
Review 10000 of 50000
Review 11000 of 50000
Review 12000 of 50000
Review 13000 of 50000
Review 14000 of 50000
Review 15000 of 50000
Review 16000 of 50000
Review 17000 of 50000
Review 18000 of 50000
Review 19000 of 50000
Review 20000 of 50000
Review 21000 of 50000
Review 22000 of 50000
Review 23000 of 50000
Review 24000 of 50000
Review 25000 of 50000
Review 26000 of 50000
Review 27000 of 50000
Review 28000 of 50000
Review 29000 of 50000
Review 30000 of 50000
Review 31000 of 50000
Review 32000 of 50000
Review 33000 of 50000
Review 34000 of 50000
Review 35000 of 50000
Review 36000 of 50000
Review 37000 of 50000
Review 38000 of 50000
Review 39000 of 50000
Review 40000 of 50000
Review 41000 of 50000
Review 42000 of 50000
Review 43000 of 50000
Review 44000 of 50000
Review 45000 of 50000
Review 46000 of 500

In [28]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
    
print("Fitting random forest to training data....")    
forest = forest.fit(trainDataVecs, df["sentiment"])

Fitting random forest to training data....


In [31]:
result = forest.predict(trainDataVecs)

In [34]:
Y_ = df['sentiment'].values

In [37]:
Y_ = Y_.reshape(Y_.shape[0],1)

In [43]:
result = result.reshape(result.shape[0],1)

In [49]:
def score(y1, y2):
    le = y1.shape[0]
    if le == y2.shape[0]:
        er = y1 - y2
        er = er*er
        toter = np.sum(er)
        return (le - toter)/le
    else:
        print('Input must be the same dimenssion')

In [51]:
result

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])